# HR Churn model feed

This notebook will feed data to the employee attrition model you created earlier, so it can be monitored by Watson OpenScale.

You will need to create service credentials for your instance of Watson Machine Learning, which you can find on the [Cloud resources list](https://cloud.ibm.com/resources).

In [ ]:
WML_CREDENTIALS = {
  "apikey": "xxxxxxxxxxxx",
  "iam_apikey_description": "Auto-generated for key cdaxxxxx",
  "iam_apikey_name": "name",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::",
  "instance_id": "xxxxxxxx",
  "url": "https://us-south.ml.cloud.ibm.com"
}

You will also need a Cloud API key to communicate with OpenScale's feedback data endpoint. You can create one [here](https://cloud.ibm.com/iam/apikeys).

In [ ]:
CLOUD_API_KEY = "xxxxxxxxxxx"

Next, you'll need to import the "HR Gold v03.csv" file from your project using the "Find and add data" button at the top right of the menu. Insert it as a pandas DataFrame.

Drop unnecessary columns from the dataframe.

In [ ]:
df_data = df_data_1.drop(['FirstName', 'LastName', 'Country', 'StreetAddress', 'City', 'Zipcode', 'Longitude', 'Latitude', 'Attrition'], axis=1)
df_data.head()

In [ ]:
MODEL_NAME = "HR Churn"
DEPLOYMENT_NAME = "HR Churn - Production"

MIN_PAYLOADS = 100
MAX_PAYLOADS = 150

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [ ]:
churn_deployment_id = None
model_deployment_ids = client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deployment id:', deployment_id)
        churn_deployment_id = deployment_id
        print('Model id:', model_id)
        break

In [ ]:
deployment_details = client.deployments.get_details(churn_deployment_id)
scoring_url = deployment_details['entity']['scoring_url']
scoring_url

In [ ]:
import json

data = json.loads(df_data.to_json(orient='split',index=False))
data

In [ ]:
scoring_payload = {
    "fields": data['columns'],
    "values": []
}

import random

for _ in range(0, random.randint(MIN_PAYLOADS, MAX_PAYLOADS)):
    value_to_score = random.choice(data['data'])
    scoring_payload['values'].append(value_to_score)

In [ ]:
scoring_payload

In [ ]:
predictions = client.deployments.score(scoring_url, scoring_payload)

In [ ]:
predictions

In [ ]:
FEEDBACK_RECORDS = 50

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

In [ ]:
from ibm_ai_openscale import APIClient
import requests
from ibm_ai_openscale.utils import get_instance_guid

WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for sub in subscriptions_uids:
    if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
        subscription = ai_client.data_mart.subscriptions.get(sub)

In [ ]:
subscription.get_details()

In [ ]:
feedback_df = df_data_1.drop(['FirstName', 'LastName', 'Country', 'StreetAddress', 'City', 'Zipcode', 'Longitude', 'Latitude'], axis=1)
df1 = feedback_df.pop('Attrition')
feedback_df['Attrition'] = df1
feedback_set = feedback_df.values.tolist()

In [ ]:
feedback_df.head()

In [ ]:
feedback_payload = []
for _ in range(0, FEEDBACK_RECORDS):
    feedback_payload.append(random.choice(feedback_set))
print(feedback_payload[0])

In [ ]:
subscription.feedback_logging.store(feedback_payload)